# Neurocombat TOPMRI and EDIS harmonized datasets

Note this must be run in the `neuron` or `neuron_go` environment

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np


sys.path.insert(0, '../../')  # path to functions
import cvasl.vendor.neurocombat.neurocombat as neurocombat
import cvasl.harmony as har


## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
# Datasets for this work

EDE = pd.read_csv('../new_data/TrainingDataComplete_EDIS.csv')

#SABRE = pd.read_csv('../new_data/TrainingDataComplete_SABRE.csv')
MRI = pd.read_csv('../new_data/TrainingDataComplete_StrokeMRI.csv')
TOP = pd.read_csv('../new_data/TrainingDataComplete_TOP.csv')
#INSI = pd.read_csv('../new_data/TrainingDataComplete_Insight46.csv')

In [ ]:
MRI = MRI.drop(['Site', 'ID'],axis = 1)
MRI.tail(3)

In [ ]:
TOP = TOP.drop(['Site', 'ID'],axis = 1)
TOP.tail(3)

In [ ]:
TOPMRI = pd.concat([MRI ,TOP])
TOPMRI.head(3)

In [ ]:
EDE = EDE.drop(['Site', 'ID'],axis = 1)
EDE.tail(3)

In [ ]:
datasets1 = [TOPMRI,  EDE, ]
datasets = []
for myset in datasets1:
    myset = myset.drop(['LD', 'PLD', 'Labelling', 'Readout', 'M0'], axis=1)
    datasets.append(myset)
print(len(datasets1[0]))     
print(len(datasets1[1]))     
   
TOPMRI = datasets[0]
EDE   = datasets[1]
print(len(TOPMRI))     
print(len(EDE))     

In [ ]:
TOPMRI.shape

In [ ]:
TOPMRI.columns

In [ ]:
to_be_harmonized_or_covar = [
    'Age', 'Sex','DeepWM_B_CoV', 'ACA_B_CoV', 'MCA_B_CoV', 'PCA_B_CoV', 'TotalGM_B_CoV',
    'DeepWM_B_CBF', 'ACA_B_CBF', 'MCA_B_CBF', 'PCA_B_CBF', 'TotalGM_B_CBF',
]
not_harmonized= ['GM_vol', 'WM_vol', 'CSF_vol','GM_ICVRatio', 'GMWM_ICVRatio', 'WMHvol_WMvol', 'WMH_count',]
TOPMRI_semi_features = TOPMRI.drop(to_be_harmonized_or_covar,axis=1)
#HELIUS_semi_features = HELIUS.drop(to_be_harmonized_or_covar,axis=1)
EDIS_semi_features = EDE.drop(to_be_harmonized_or_covar,axis=1) 

TOPMRI = TOPMRI.drop(not_harmonized,axis=1)
#HELIUS = HELIUS.drop(not_harmonized,axis=1)
EDE = EDE.drop(not_harmonized,axis=1) 

In [ ]:
TOPMRI.shape

In [ ]:
both_togetherF, ftF, btF, feature_dictF, len1, len2 = har.prep_for_neurocombat(TOPMRI, EDE)

In [ ]:
#both_togetherF.head(3)

In [ ]:
ftF.to_csv('ftF_top_mri_insight.csv')

In [ ]:
data = np.genfromtxt('ftF_top_mri_insight.csv', delimiter=",", skip_header=1)
data = data[:, 1:]
data

In [ ]:
TOPMRI.head(3)

In [ ]:
both_togetherF.head(3)

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len1
last_columns_as_two = [2] * len2
covars = {'batch':first_columns_as_one + last_columns_as_two,
          #'Site': both_togetherF.loc['Site',:].values.tolist(), 
          'Sex':both_togetherF.loc['Sex',:].values.tolist(),
           'Age':both_togetherF.loc['Age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
# specifify sex as categorical
categorical_cols = ['Sex']
# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'
our_continuous_col=['Age']
#Harmonization step:
data_combat = neurocombat.neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col_mine,
    continuous_cols=our_continuous_col,
    categorical_cols=categorical_cols)["data"]

In [ ]:
neurocombat = pd.DataFrame(data_combat)
neurocombat

In [ ]:
topperF = har.make_topper(btF,'Age', 'Sex')

In [ ]:
bottom = neurocombat.reset_index(drop=False)
bottom = bottom.rename(columns={"index": "char"})
bottom.columns = topperF.columns

In [ ]:
back_together = pd.concat([topperF, bottom])
back_together = back_together.T
#back_together

In [ ]:
new_header = back_together.iloc[0] #grab the first row for the header
back_together.columns = new_header #set the header row as the df header
back_together = back_together[1:]
#back_together

In [ ]:
back_together.head(3)

In [ ]:
neuro_harm_edis =back_together.tail(len2)
neuro_harm_topmri =back_together.head(len1)

In [ ]:
print(neuro_harm_edis.shape)
print(neuro_harm_topmri.shape)

In [ ]:
neuro_harm_topmri_v_e = neuro_harm_topmri.rename(feature_dictF, axis='columns')
neuro_harm_edis = neuro_harm_edis.rename(feature_dictF, axis='columns')

In [ ]:
neuro_harm_edis.shape

In [ ]:
neuro_harm_edis = neuro_harm_edis.reset_index()
neuro_harm_edis = neuro_harm_edis.rename(columns={"index": "participant_id"})
neuro_harm_edis.shape

Save off to harmonized csv files

In [ ]:
#neuro_harm_edis = neuro_harm_edis.reset_index()
neuro_harm_edis = neuro_harm_edis.merge(EDIS_semi_features,on="participant_id")
neuro_harm_edis.shape

In [ ]:
neuro_harm_topmri_v_e = neuro_harm_topmri_v_e.reset_index()
neuro_harm_topmri_v_e = neuro_harm_topmri_v_e.rename(columns={"index": "participant_id"})
neuro_harm_topmri_v_e.shape

In [ ]:

neuro_harm_topmri_v_e = neuro_harm_topmri_v_e.merge(TOPMRI_semi_features,on="participant_id")
neuro_harm_topmri_v_e.shape

In [ ]:
print(neuro_harm_edis.shape)
print(neuro_harm_topmri_v_e.shape)

In [ ]:
print(neuro_harm_edis.columns)
print(neuro_harm_topmri_v_e.columns)

In [ ]:
neuro_harm_edis.to_csv('harm_results/neuro_harm_edis.csv')
neuro_harm_topmri_v_e.to_csv('harm_results/neuro_harm_topmri_v_e.csv')

In [ ]:
#neuro_harm_sabre.columns

In [ ]:
number_columns = ['Age', 'Sex', 'GM_vol', 'WM_vol', 'CSF_vol',
       'GM_ICVRatio', 'GMWM_ICVRatio', 'WMHvol_WMvol', 'WMH_count',
    'ACA_B_CoV', 'MCA_B_CoV', 'PCA_B_CoV', 'TotalGM_B_CoV',
    'ACA_B_CBF', 'MCA_B_CBF', 'PCA_B_CBF', 'TotalGM_B_CBF',]

# Data quality check

In [ ]:
TOPMRI

In [ ]:
neuro_harm_topmri_v_e

In [ ]:
neuro_harm_edis.isna().sum().sum()

In [ ]:
neuro_harm_topmri.isna().sum().sum()

In [ ]:
#neuro_harm_top.head(3)
(neuro_harm_edis[number_columns] < 0).sum()

In [ ]:
(neuro_harm_topmri_v_e[number_columns] < 0).sum()

 We will wait about logging it until we see other columns we may want to log across all datasets